# Import Packages

In [ ]:
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import csv

# Import Data

In [ ]:
dat = pd.read_csv('../data/pubmed.csv')

# Generate unseen labels

In [ ]:
x = list(dat['sequence'])

yy = []
for i in range(len(x)):
    temp = x[i].replace("[","").replace("]","").replace("(","").replace(")","").replace("'","").split(", ")
    y = []
    for j in range(len(temp)):
        if j%2 != 0:
            y.append(temp[j])
    yy.append(y)
    
dat['mesh term code'] = yy

dct_txt_cd = {}
dct_cd_txt = {}
for i in range(len(x)):
    temp = x[i].replace("[","").replace("]","").replace("(","").replace(")","").replace("'","").split(", ")
    for j in range(len(temp)):
        if j%2 != 1:
            if temp[j] not in dct_txt_cd:
                dct_txt_cd[temp[j]] = temp[j+1]
                dct_cd_txt[temp[j+1]] = temp[j]
                
temp_code = dat['mesh term code']

dct = {}
for i in range(len(temp_code)):
    for j in range(len(temp_code[i])):
        if temp_code[i][j] not in dct:
            dct[temp_code[i][j]] = 1
        else:
            dct[temp_code[i][j]] += 1
            
mesh_list = pd.DataFrame.from_dict(dct, orient='index')
mesh_list['density'] = mesh_list[0]/np.shape(dat)[0]
rm_list = mesh_list[mesh_list['density'] < 0.001].sample(frac = 0.2, random_state = 3)['density']

#rm_labels.csv Output file
rm_list.to_csv('../data/rm_labels.csv')